In [1]:
import numpy as np

import dataset

import cv2

In [2]:
HEIGHT = 64
WIDTH = 64
DEPTH = 3

DATA_PATH = '../dataset/stl10_binary/train_X.bin'
LABEL_PATH = '../dataset/stl10_binary/train_y.bin'

In [23]:
images = dataset.read_all_images(DATA_PATH)
labels = dataset.read_labels(LABEL_PATH)

In [24]:
def get_resized_images(images):
    train_X=np.empty((images.shape[0],HEIGHT,WIDTH,3),dtype=np.uint8)
    
    for i in range(images.shape[0]):
        train_X[i] = cv2.resize(images[i],(HEIGHT,WIDTH))
    
    return train_X

In [25]:
train_X = get_resized_images(images)

(5000, 64, 64, 3)